In [2]:
# Imports
%matplotlib inline


from My_Functions import * 
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from collections import defaultdict

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score,roc_curve
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict,validation_curve
from sklearn.ensemble import RandomForestRegressor


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler,PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator , MultipleLocator
##from matplotlib import XAxis
from gplearn.genetic import SymbolicRegressor

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression,LinearRegression, OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split , TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from matplotlib.gridspec import GridSpec
import plotly.tools as tls
import plotly
plotly.tools.set_credentials_file(username='Furqan92', api_key='22DfVN5rFRg79OYygN5h')
import plotly.plotly as py
from sklearn.decomposition import PCA
from pandas import DataFrame 
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action='ignore')
random_seed = 1234
tscv = TimeSeriesSplit(n_splits=5)

In [3]:
# Load datasets
we_h=pd.read_csv('./weekends_data_prepared.csv')
we_h['dteday']=pd.to_datetime(we_h['dteday'], format='%Y-%m-%d')

## Subset 2011 -> 2012Q3 Data

In [4]:
X_Train_2011_2012Q2 = we_h[we_h['dteday'] < datetime.datetime(2012, 7, 1, 0, 0)].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THst_EM IN DATA
Y_cnt_train_2011_2012Q2 =we_h['cnt'][we_h['dteday'] < datetime.datetime(2012, 7, 1, 0, 0)]

X_Test_2012Q3 = we_h[(we_h['dteday'] >= datetime.datetime(2012, 7, 1, 0, 0)) & (we_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_2012Q3 =we_h['cnt'][(we_h['dteday'] >= datetime.datetime(2012, 7, 1, 0, 0)) & (we_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))]


## Linear regression w/ GS for *Weekdays* for 2011 -> 2012Q3

In [5]:
##Linear Regression
lm_parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv,return_train_score=True)

lm.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# lm.cv_results_
lm_predictions = lm.predict(X_Test_2012Q3)
lm.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

0.8231899395670057

## Random Forest w/ GS for *weekends* for 2011 -> 2012Q3

In [6]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

rf.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# rf.cv_results_
rf_predictions = lm.predict(X_Test_2012Q3)
rf.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

0.8580376554177968

## Taking predictions from LR and Random Forest to test on 2012Q4

In [7]:
# LR and Random Forest predictions
combinedPredictions = pd.DataFrame({'lm_predictions':lm_predictions,'rf_predictions':rf_predictions})

# Concat original data
X_Test_2012Q3.reset_index(drop=True,inplace=True)
combinedPredictions = pd.concat([combinedPredictions,X_Test_2012Q3], axis=1)

# Target data
combinedPredictionsTarget = pd.DataFrame({'target':Y_cnt_test_2012Q3})

#Reset indices
combinedPredictions.reset_index(drop=True,inplace=True)
combinedPredictionsTarget.reset_index(drop=True,inplace=True)

## Subset Q4 data and concat it with its level-0 predctions

In [29]:
#Get Q4 data
X_Test_Q4 = we_h[(we_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (we_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_Q4 =we_h['cnt'][(we_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (we_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))]
oringal_cnt = Y_cnt_test_Q4

#get level-0 predictions for Q4 data
Q4_lm_predications = pd.DataFrame(lm.predict(X_Test_Q4), columns=["lm_predications"])
Q4_rf_predications = pd.DataFrame(rf.predict(X_Test_Q4), columns=["rf_predications"])

X_Test_Q4.reset_index(drop=True,inplace=True)
Q4_lm_predications.reset_index(drop=True,inplace=True)
Q4_rf_predications.reset_index(drop=True,inplace=True)
Y_cnt_test_Q4.reset_index(drop=True,inplace=True)

#concat Q4 with level-0 predictions 
X_Test_Q4_with_predictions = pd.concat([Q4_lm_predications, Q4_rf_predications, X_Test_Q4], axis=1)
Y_cnt_test_Q4 = pd.DataFrame({"target": Y_cnt_test_Q4})

## Q4 scores

In [9]:
lm.score(X_Test_Q4, Y_cnt_test_Q4)

0.8558632250118965

In [10]:
rf.score(X_Test_Q4, Y_cnt_test_Q4)

0.8603852437202893

## Train and test Q4

In [11]:

RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                            'criterion':['mse']}
rf_level_1 = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

rf_level_1.fit(combinedPredictions, combinedPredictionsTarget)
rf_level_1.cv_results_
rf_level_1_predictions = lm.predict(X_Test_Q4)
rf_level_1.score(X_Test_Q4_with_predictions, Y_cnt_test_Q4)

0.8569059192242999

In [12]:
target = pd.DataFrame({"predicted_lvl_1_cnt": rf_level_1_predictions})

X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred = pd.concat([X_Test_Q4_with_predictions, oringal_cnt, target], axis=1)
X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred



,lm_predications,rf_predications,windspeed,yr,temp,RushHour-Low,hum,relative_windspeed,relative_hum,atemp,...,7,8,9,10,11,12,13,14,cnt,predicted_lvl_1_cnt
0,141.101318,111.947472,0.1642,1,0.56,1,0.83,0.436246,0.860341,0.5303,...,1.120302,1.832895,3.660302,3.660302,3.660302,1.660302,2.660302,2.100302,191,141.101318
1,121.213134,96.651949,0.2537,1,0.56,1,0.83,1.351133,0.851026,0.5303,...,1.120302,1.832895,3.660302,3.660302,3.660302,1.660302,2.660302,2.100302,141,121.213134
2,114.944091,88.432555,0.1940,1,0.56,1,0.78,0.740650,0.547044,0.5303,...,1.220302,1.882895,3.660302,3.660302,3.660302,1.660302,2.660302,2.100302,75,114.944091
3,87.230142,56.000733,0.2239,1,0.54,1,0.77,1.047542,0.480767,0.5152,...,1.160302,1.832895,3.620302,3.620302,3.620302,1.620302,2.620302,2.080302,56,87.230142
4,67.550455,18.359926,0.2537,1,0.54,1,0.83,1.346404,0.828718,0.5152,...,1.080302,1.772895,3.620302,3.620302,3.620302,1.620302,2.620302,2.080302,11,67.550455
5,67.320474,22.336203,0.1940,1,0.54,1,0.83,0.740650,0.819479,0.5152,...,1.080302,1.772895,3.620302,3.620302,3.620302,1.620302,2.620302,2.080302,10,67.320474
6,73.040445,42.084323,0.2537,1,0.54,1,0.88,1.346404,1.105496,0.5152,...,1.080302,1.722895,3.620302,3.620302,3.620302,1.620302,2.620302,2.080302,30,73.040445
7,78.686685,74.162845,0.2985,1,0.54,1,0.83,1.800973,0.799834,0.5152,...,1.080302,1.772895,3.620302,3.620302,3.620302,1.620302,2.620302,2.080302,84,78.686685
8,240.695657,197.813719,0.1940,1,0.56,0,0.83,0.738598,0.792786,0.5303,...,3.580000,3.850000,5.580000,5.580000,5.580000,3.850000,4.580000,3.290000,206,240.695657
9,327.246259,270.109846,0.1940,1,0.60,0,0.73,0.748882,0.201486,0.5909,...,3.940000,4.070000,5.940000,5.940000,5.940000,4.070000,4.940000,3.470000,395,327.246259


In [13]:
X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred.columns

Index(['lm_predications', 'rf_predications', 'windspeed', 'yr', 'temp',
       'RushHour-Low', 'hum', 'relative_windspeed', 'relative_hum', 'atemp',
       'workingday', 'isNoon', 'isDaylight', 'relative_temp', 'holiday',
       'relative_atemp', 'RushHour-Med', 'RushHour-High', 's_fall', 's_spring',
       's_summer', 's_winter', 'w_Acceptable', 'w_Bad', 'w_Chaos', 'w_Good',
       'm_1', 'm_2', 'm_3', 'm_4', 'm_5', 'm_6', 'm_7', 'm_8', 'm_9', 'm_10',
       'm_11', 'm_12', 'w_0', 'w_6', 'h_0', 'h_1', 'h_2', 'h_3', 'h_4', 'h_5',
       'h_6', 'h_7', 'h_8', 'h_9', 'h_10', 'h_11', 'h_12', 'h_13', 'h_14',
       'h_15', 'h_16', 'h_17', 'h_18', 'h_19', 'h_20', 'h_21', 'h_22', 'h_23',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', 'cnt', 'predicted_lvl_1_cnt'],
      dtype='object')

In [59]:
level_1=pd.DataFrame(X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred['predicted_lvl_1_cnt'])
level_1.reset_index(drop=True,inplace=True)

In [60]:
level_0=pd.DataFrame(X_Test_Q4_with_lv  l_0_pred_and_lvl_1_pred['lm_predications'])
level_0.reset_index(drop=True,inplace=True)

In [70]:
actual=pd.DataFrame(X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred['cnt'])
actual.reset_index(drop=True,inplace=True)

In [80]:
combined=pd.concat([level_0,level_1,actual],axis=1)


In [72]:
dates=pd.DataFrame(we_h['dteday'])

In [73]:
dates=dates[dates['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)]

In [74]:
dates.reset_index(drop=True,inplace=True)
combined.reset_index(drop=True,inplace=True)

In [82]:
full_we=pd.concat([dates,combined],axis=1)

In [83]:
full_we

,dteday,lm_predications,predicted_lvl_1_cnt,cnt
0,2012-10-06,141.101318,141.101318,191
1,2012-10-06,121.213134,121.213134,141
2,2012-10-06,114.944091,114.944091,75
3,2012-10-06,87.230142,87.230142,56
4,2012-10-06,67.550455,67.550455,11
5,2012-10-06,67.320474,67.320474,10
6,2012-10-06,73.040445,73.040445,30
7,2012-10-06,78.686685,78.686685,84
8,2012-10-06,240.695657,240.695657,206
9,2012-10-06,327.246259,327.246259,395


In [87]:
full_we.to_csv('/Users/furqan/Documents/GitHub/Python_group_project/Bike-Sharing-Dataset 2/full_we.csv',sep=',')